In [4]:
def load_urls_from_file(file_path):
    with open(file_path, 'r') as file:
        urls = [line.strip() for line in file.readlines()]
    return urls

url_list = load_urls_from_file("relative_path_to_file_containing_urls")

In [6]:
import os
import pydub
from pydub import AudioSegment

transcript_directory = "relative_path_to_directory_containing_transcripts"
audio_directory = "relative_path_to_directory_containing_audio_files"

for i in range(len(url_list)):
    os.makedirs(audio_directory + str(i) + "/sentenceFiles" , exist_ok=True)

def split_audio_file(input_file, output_file, start_time, duration):
    audio = AudioSegment.from_file(input_file)
    start_time_ms = start_time * 1000  # Convert start time to milliseconds
    end_time_ms = start_time_ms + (duration * 1000)  # Calculate end time in milliseconds
    output_audio = audio[start_time_ms:end_time_ms]  # Extract the desired segment

    output_audio.export(output_file, format="wav")  # Save the segment as a new audio file


for i in range(len(url_list)):
    t_dir = transcript_directory + str(i) + "/en.vtt"
    
    for file in os.listdir(audio_directory + str(i) + "/"):
        if file.endswith(".wav"):
            input_file = audio_directory + str(i) + "/" + file
            break
    
    save_directory = audio_directory + str(i) + "/sentenceFiles/"

    with open(t_dir, 'r', encoding='utf-8') as file:
        lines = file.readlines()
        for line in lines:
            if line[0] == '0':
                # Extract the start and end time stamps from this line
                start_time = line[0:12]
                end_time = line[17:29]
                # Convert the time stamps to seconds
                start_time = int(start_time[0:2])*3600 + int(start_time[3:5])*60 + int(start_time[6:8]) + int(start_time[9:12])/1000
                end_time = int(end_time[0:2])*3600 + int(end_time[3:5])*60 + int(end_time[6:8]) + int(end_time[9:12])/1000

                effective_start_time = start_time - 6
                effective_end_time = end_time - 6

                output_file = save_directory + str(i) + "_" + str(start_time) + "_" + str(end_time) + ".wav"
                duration = end_time - start_time
                split_audio_file(input_file, output_file, effective_start_time, duration)